In [ ]:
!pip install idx2numpy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import idx2numpy

In [ ]:
x_train_file = '/content/drive/MyDrive/emnist/emnist-letters-train-images-idx3-ubyte'
y_train_file = '/content/drive/MyDrive/emnist/emnist-letters-train-labels-idx1-ubyte'
x_test_file = '/content/drive/MyDrive/emnist/emnist-letters-test-images-idx3-ubyte'
y_test_file = '/content/drive/MyDrive/emnist/emnist-letters-test-labels-idx1-ubyte'
#위의 코드에서 에러 발생시 아래 코드의 주석을 지우고 실행
#x_train_file = '/content/Drive/MyDrive/emnist/emnist-letters-train-images-idx3-ubyte'
#y_train_file = '/content/Drive/MyDrive/emnist/emnist-letters-train-labels-idx1-ubyte'
#x_test_file = '/content/Drive/MyDrive/emnist/emnist-letters-test-images-idx3-ubyte'
#y_test_file = '/content/Drive/MyDrive/emnist/emnist-letters-test-labels-idx1-ubyte'

x_train = idx2numpy.convert_from_file(x_train_file)
y_train = idx2numpy.convert_from_file(y_train_file)
x_test = idx2numpy.convert_from_file(x_test_file)
y_test = idx2numpy.convert_from_file(y_test_file)

In [ ]:
print(x_train.shape)
print(x_test.shape)

In [ ]:
print("Y[0] : ",  y_train[0])
plt.imshow(x_train[0], cmap=plt.cm.gray_r, interpolation = "nearest")

In [ ]:
x_train=x_train/255
x_test=x_test/255

#array NumPy
x_train = np.array(x_train).reshape(-1, 28, 28, 1)
x_test =  np.array(x_test).reshape(-1, 28, 28, 1)
print(x_train.shape)
print(x_test.shape)


y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)
y_train = y_train[..., :26]
y_test = y_test[..., :26]

In [ ]:
#y_train = tf.keras.utils.to_categorical(y_train)
#y_test = tf.keras.utils.to_categorical(y_test)

#y_train = y_train[..., :26]
#y_test = y_test[..., :26]

In [ ]:
cnn= tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='tanh', input_shape=(28, 28, 1)))
cnn.add(tf.keras.layers.MaxPooling2D(strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=48, kernel_size=(5,5), padding='same', activation='tanh'))
cnn.add(tf.keras.layers.MaxPooling2D( strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(5,5), padding='same', activation='tanh'))
cnn.add(tf.keras.layers.Flatten())
# cnn.add(tf.keras.layers.Dense(512, activation='tanh'))
# cnn.add(tf.keras.layers.Dense(84, activation='tanh'))
cnn.add(tf.keras.layers.Dense(26, activation='softmax'))

#optimazer
opt=tf.keras.optimizers.Adam(learning_rate=1e-4)

#Let's compile the model before training.
cnn.compile(optimizer=opt,
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

cnn.summary()

tf.keras.utils.plot_model(cnn, "model.png", show_shapes=True)

In [ ]:
print(cnn.output_shape)

In [ ]:
keras_callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, min_delta=0.001)
]

history = cnn.fit(x_train, y_train, epochs=100, batch_size=64, verbose=1,
                    validation_split=0.2, callbacks=keras_callbacks)

In [ ]:
cnn.evaluate(x_test, y_test, verbose=2)

In [ ]:
#양자화없이 Lite로 변환
converter = tf.lite.TFLiteConverter.from_keras_model(cnn)
tflite_model = converter.convert()
open("model.tflite", "wb").write(tflite_model)
#양자화해서 Lite로 변환
converter = tf.lite.TFLiteConverter.from_keras_model(cnn)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_model = converter.convert()
open("model_quantized.tflite", "wb").write(tflite_model)

In [ ]:
# xxd를 사용할 수 없을 경우, 설치한다.
!apt-get -qq install xxd
# 파일을 C 소스파일로 저장
!xxd -i model.tflite > model.cc
# 소스파일을 출력
!cat model.cc
# 파일을 C 소스파일로 저장
!xxd -i model_quantized.tflite > model_quantized.cc
# 소스파일을 출력
!cat model_quantized.cc